# Ray Tracing: Is our analytical approach going to work?

# Fourth question: Should we care about tremor streaks?

If we denote $t_d$ the arrival time of the direct wave, and $t_r$ the arrival time of the reflected wave, the time lag
between the two phase arrivals is $tlag = t_d - t_r$. During the one-minute time window where we compute the auto / cross correlation, the displacement $dx$ of the tremor source along the plate boundary should corresponds to a time lag difference $dtlag$ shorter than a quarter of the dominant period of $T$ = 0.3 s. During an ETS event, rapid tremor
streaks have been observed propagating in the up-dip and down-dip directions at velocities ranging between 30 and 110 km/h, and possibly up to 200 km/h, which corresponds to a source displacement of 0.5 to 1.8 km, and up to 3.3 km during one minute.

We compute the time lag difference for a displacement of the source of 0.5, 1.8 and 3.3 km in the up-dip direction, for stations aligned along the strike and along the dip direction.

We compute only the time lag between the arrival of the direct P/S wave and the arrival of the phases with amplitude ratio between 0.5 and 2.

## Python code

Load Python modules.

In [1]:
import numpy
from math import sqrt, pi, cos, sin, tan

Load functions from my own ray-tracing related modules.

In [2]:
from computeAmplitude import computeAmplitude3SH, computeAmplitude5SH, computeAmplitude3PSV, computeAmplitude5PSV
from computeAngle import computeAngle3, computeAngle5
from computeTravelTime import computeTravelTime3, computeTravelTime5
from misc import computeInitAmp

Get P- and S-wave velocities in the continental crust.

In [3]:
from data import VpCC, VsCC

Get strike and dip of the subducted oceanic plate.

In [4]:
from data import phi, delta

Stations aligned along strike (distances in m).

In [5]:
x1 = sin(phi * pi / 180.0) * numpy.arange(- 20000.0, 22000.0, 2000.0)
y1 = cos(phi * pi / 180.0) * numpy.arange(- 20000.0, 22000.0, 2000.0)

Stations aligned along dip (distances in m).

In [6]:
x2 = cos(phi * pi / 180.0) * numpy.arange(- 20000.0, 22000.0, 2000.0)
y2 = - sin(phi * pi / 180.0) * numpy.arange(- 20000.0, 22000.0, 2000.0)

Source:

Propagation of tremors along dip at 30 / 110 / 200 km/h.

-> Displacement of source in 1 min is close to: 500 / 1800 / 3300 m.

In [7]:
x_s = - cos(phi * pi / 180.0) * numpy.array([0.0, 500.0, 1800.0, 3300.0])
y_s = sin(phi * pi / 180.0) * numpy.array([0.0, 500.0, 1800.0, 3300.0])
d_s = 35000.0 - tan(delta * pi /180.0) * numpy.array([0.0, 500.0, 1800.0, 3300.0])

Types of wave.

In [8]:
wave = ('P', 'S')

Initializations.

In [9]:
tP = numpy.zeros((numpy.shape(x_s)[0]))
tS = numpy.zeros((numpy.shape(x_s)[0]))
t3SH = numpy.zeros((numpy.shape(x_s)[0]))
t5SH = numpy.zeros((numpy.shape(x_s)[0]))
t3PSV = numpy.zeros((numpy.shape(x_s)[0], 8))
t5PSV = numpy.zeros((numpy.shape(x_s)[0], 32))
A3PSV = numpy.zeros(8)
A5PSV = numpy.zeros(32)

Open output file.

In [10]:
output = open('TimeDifferenceStrikeDip.txt', 'w')

Loop on stations along strike.

In [11]:
output.write('---------------------\n')
output.write('Stations along strike\n')
output.write('---------------------\n')
for i in range(0, numpy.shape(x1)[0]):
    output.write('\n')
    output.write('Station {} at {} km from the tremor epicenter\n'.format(i + 1, sqrt(x1[i]**2 + y1[i]**2)))
    output.write('\n')
    # Loop on source position
    for j in range(0, numpy.shape(x_s)[0]):
        # Direct wave
        tP[j] = sqrt((x1[i] - x_s[j]) ** 2.0 + (y1[i] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VpCC
        tS[j] = sqrt((x1[i] - x_s[j]) ** 2.0 + (y1[i] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VsCC
        if j == 0:
            AD = computeInitAmp(x_s[j], y_s[j], x1[i], y1[i], d_s[j], 'D')
            AP = AD[0]
            ASV = AD[1]
            ASH = AD[2]
        # Reflected SH-wave on mid-slab discontinuity
        angle = computeAngle3(x_s[j], y_s[j], d_s[j], x1[i], y1[i], 'S', 'S', 'S')
        t3SH[j] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x1[i], y1[i], 'S', 'S', 'S')
        if j == 0:
            A3SH = computeAmplitude3SH(angle, x_s[j], y_s[j], x1[i], y1[i])
            AR = computeInitAmp(x_s[j], y_s[j], x1[i], y1[i], d_s[j], 'R', angle)
            A3SH = A3SH * AR[2]
        # Reflected SH-wave on Moho
        angle = computeAngle5(x_s[j], y_s[j], d_s[j], x1[i], y1[i], 'S', 'S', 'S', 'S', 'S')
        t5SH[j] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x1[i], y1[i], 'S', 'S', 'S', 'S', 'S')
        if j == 0:
            A5SH = computeAmplitude5SH(angle, x_s[j], y_s[j], x1[i], y1[i])
            AR = computeInitAmp(x_s[j], y_s[j], x1[i], y1[i], d_s[j], 'R', angle)
            A5SH = A5SH * AR[2]
        # Reflected wave on mid-slab discontinuity
        # Downgoing wave in upper oceanic crust
        for k1 in range(0, 2):
            # Upgoing wave in upper oceanic crust
            for k2 in range(0, 2):
                # Upgoing wave in continental crust
                for k3 in range(0, 2):
                    k = k1 * 1 + k2 * 2 + k3 * 4
                    angle = computeAngle3(x_s[j], y_s[j], d_s[j], x1[i], y1[i], wave[k1], wave[k2], wave[k3])
                    t3PSV[j, k] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x1[i], y1[i], \
                                  wave[k1], wave[k2], wave[k3])
                    if j == 0:
                        A3PSV[k] = computeAmplitude3PSV(angle, x_s[j], y_s[j], x1[i], y1[i], \
                                   wave[k1], wave[k2], wave[k3])
                        AR = computeInitAmp(x_s[j], y_s[j], x1[i], y1[i], d_s[j], 'R', angle)
                        if (k1 == 0):
                            A3PSV[k] = A3PSV[k] * AR[0]
                        else:
                            A3PSV[k] = A3PSV[k] * AR[1]
        # Reflected wave on Moho
        # Downgoing wave in upper oceanic crust
        for k1 in range(0, 2):
            # Downgoing wave in lower oceanic crust
            for k2 in range(0, 2):
                # Upgoing wave in lower oceanic crust
                for k3 in range(0, 2):
                    # Upgoing wave in upper oceanic crust
                    for k4 in range(0, 2):
                        # Upgoing wave in continental crust
                        for k5 in range(0, 2):
                            k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                            angle = computeAngle5(x_s[j], y_s[j], d_s[j], x1[i], y1[i], \
                                    wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                            t5PSV[j, k] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x1[i], y1[i], \
                                          wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                            if j == 0:
                                A5PSV[k] = computeAmplitude5PSV(angle, x_s[j], y_s[j], x1[i], y1[i], \
                                           wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                                AR = computeInitAmp(x_s[j], y_s[j], x1[i], y1[i], d_s[j], 'R', angle)
                                if (k1 == 0):
                                    A5PSV[k] = A5PSV[k] * AR[0]
                                else:
                                    A5PSV[k] = A5PSV[k] * AR[1]

    output.write('Time lag between direct waves\n')
    if ((abs(ASV / AP) >= 0.5) and (abs(ASV / AP) <= 2.0)):
        output.write('Amplitude of direct SV-wave: {}\n'.format(ASV / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (tS[1] - tP[1]) - (tS[0] - tP[0]), \
            (tS[2] - tP[2]) - (tS[0] - tP[0]), \
            (tS[3] - tP[3]) - (tS[0] - tP[0])))
        output.write('\n')
    if ((abs(ASH / AP) >= 0.5) and (abs(ASH / AP) <= 2.0)):
        output.write('Amplitude of direct SH-wave: {}\n'.format(ASH / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (tS[1] - tP[1]) - (tS[0] - tP[0]), \
            (tS[2] - tP[2]) - (tS[0] - tP[0]), \
            (tS[3] - tP[3]) - (tS[0] - tP[0])))
        output.write('\n')

    output.write('Time lag with direct P-wave\n')
    if ((abs(A3SH / AP) >= 0.5) and (abs(A3SH / AP) <= 2.0)):
        output.write('Amplitude of SH-wave (mid-slab): {}\n'.format(A3SH / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t3SH[1] - tP[1]) - (t3SH[0] - tP[0]), \
            (t3SH[2] - tP[2]) - (t3SH[0] - tP[0]), \
            (t3SH[3] - tP[3]) - (t3SH[0] - tP[0])))
        output.write('\n')
    if ((abs(A5SH / AP) >= 0.5) and (abs(A5SH / AP) <= 2.0)):
        output.write('Amplitude of SH-wave (Moho): {}\n'.format(A5SH / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t5SH[1] - tP[1]) - (t5SH[0] - tP[0]), \
            (t5SH[2] - tP[2]) - (t5SH[0] - tP[0]), \
            (t5SH[3] - tP[3]) - (t5SH[0] - tP[0])))
        output.write('\n')
    # Reflected wave on mid-slab discontinuity
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Upgoing wave in upper oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in continental crust
            for k3 in range(0, 2):
                k = k1 * 1 + k2 * 2 + k3 * 4
                if ((abs(A3PSV[k] / AP) >= 0.5) and (abs(A3PSV[k] / AP) <= 2.0)):
                    output.write('Amplitude of ray {}{}{}: {}\n'.format(wave[k1], wave[k2], wave[k3], A3PSV[k] / AP))
                    output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                        (t3PSV[1, k] - tP[1]) - (t3PSV[0, k] - tP[0]), \
                        (t3PSV[2, k] - tP[2]) - (t3PSV[0, k] - tP[0]), \
                        (t3PSV[3, k] - tP[3]) - (t3PSV[0, k] - tP[0])))
                    output.write('\n')
    # Reflected wave on Moho
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Downgoing wave in lower oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in lower oceanic crust
            for k3 in range(0, 2):
                # Upgoing wave in upper oceanic crust
                for k4 in range(0, 2):
                    # Upgoing wave in continental crust
                    for k5 in range(0, 2):
                        k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                        if ((abs(A5PSV[k] / AP) >= 0.5) and (abs(A35SV[k] / AP) <= 2.0)):
                            output.write('Amplitude of ray {}{}{}{}{}: {}\n'.format( \
                                wave[k1], wave[k2], wave[k3], wave[k4], wave[k5], A5PSV[k] / AP))
                            output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                                (t5PSV[1, k] - tP[1]) - (t5PSV[0, k] - tP[0]), \
                                (t5PSV[2, k] - tP[2]) - (t5PSV[0, k] - tP[0]), \
                                (t5PSV[3, k] - tP[3]) - (t5PSV[0, k] - tP[0])))
                            output.write('\n')

    output.write('Time lag with direct S-wave\n')
    if ((abs(A3SH / ASH) >= 0.5) and (abs(A3SH / ASH) <= 2.0)):
        output.write('Amplitude of SH-wave (mid-slab): {}\n'.format(A3SH / ASH))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t3SH[1] - tS[1]) - (t3SH[0] - tS[0]), \
            (t3SH[2] - tS[2]) - (t3SH[0] - tS[0]), \
            (t3SH[3] - tS[3]) - (t3SH[0] - tS[0])))
        output.write('\n')
    if ((abs(A5SH / ASH) >= 0.5) and (abs(A5SH / ASH) <= 2.0)):
        output.write('Amplitude of SH-wave (Moho): {}\n'.format(A5SH / ASH))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t5SH[1] - tS[1]) - (t5SH[0] - tS[0]), \
            (t5SH[2] - tS[2]) - (t5SH[0] - tS[0]), \
            (t5SH[3] - tS[3]) - (t5SH[0] - tS[0])))
        output.write('\n')
    # Reflected wave on mid-slab discontinuity
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Upgoing wave in upper oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in continental crust
            for k3 in range(0, 2):
                k = k1 * 1 + k2 * 2 + k3 * 4
                if ((abs(A3PSV[k] / ASV) >= 0.5) and (abs(A3PSV[k] / ASV) <= 2.0)):
                    output.write('Amplitude of ray {}{}{}: {}\n'.format(wave[k1], wave[k2], wave[k3], A3PSV[k] / ASV))
                    output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                        (t3PSV[1, k] - tS[1]) - (t3PSV[0, k] - tS[0]), \
                        (t3PSV[2, k] - tS[2]) - (t3PSV[0, k] - tS[0]), \
                        (t3PSV[3, k] - tS[3]) - (t3PSV[0, k] - tS[0])))
                    output.write('\n')
    # Reflected wave on Moho
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Downgoing wave in lower oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in lower oceanic crust
            for k3 in range(0, 2):
                # Upgoing wave in upper oceanic crust
                for k4 in range(0, 2):
                    # Upgoing wave in continental crust
                    for k5 in range(0, 2):
                        k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                        if ((abs(A5PSV[k] / ASV) >= 0.5) and (abs(A3PSV[k] / ASV) <= 2.0)):
                            output.write('Amplitude of ray {}{}{}{}{}: {}\n'.format( \
                                wave[k1], wave[k2], wave[k3], wave[k4], wave[k5], A5PSV[k] / ASV))
                            output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                                (t5PSV[1, k] - tS[1]) - (t5PSV[0, k] - tS[0]), \
                                (t5PSV[2, k] - tS[2]) - (t5PSV[0, k] - tS[0]), \
                                (t5PSV[3, k] - tS[3]) - (t5PSV[0, k] - tS[0])))
                            output.write('\n')

Loop on stations along dip.

In [12]:
output.write('\n')
output.write('------------------\n')
output.write('Stations along dip\n')
output.write('------------------\n')
for i in range(0, numpy.shape(x2)[0]):
    output.write('\n')
    output.write('Station {} at {} km from the tremor epicenter\n'.format(i + 1, sqrt(x2[i]**2 + y2[i]**2)))
    output.write('\n')
    # Loop on source position
    for j in range(0, numpy.shape(x_s)[0]):
        # Direct wave
        tP[j] = sqrt((x2[i] - x_s[j]) ** 2.0 + (y2[i] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VpCC
        tS[j] = sqrt((x2[i] - x_s[j]) ** 2.0 + (y2[i] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VsCC
        if j == 0:
            AD = computeInitAmp(x_s[j], y_s[j], x2[i], y2[i], d_s[j], 'D')
            AP = AD[0]
            ASV = AD[1]
            ASH = AD[2]
        # Reflected SH-wave on mid-slab discontinuity
        angle = computeAngle3(x_s[j], y_s[j], d_s[j], x2[i], y2[i], 'S', 'S', 'S')
        t3SH[j] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x2[i], y2[i], 'S', 'S', 'S')
        if j == 0:
            A3SH = computeAmplitude3SH(angle, x_s[j], y_s[j], x2[i], y2[i])
            AR = computeInitAmp(x_s[j], y_s[j], x2[i], y2[i], d_s[j], 'R', angle)
            A3SH = A3SH * AR[2]
        # Reflected SH-wave on Moho
        angle = computeAngle5(x_s[j], y_s[j], d_s[j], x2[i], y2[i], 'S', 'S', 'S', 'S', 'S')
        t5SH[j] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x2[i], y2[i], 'S', 'S', 'S', 'S', 'S')
        if j == 0:
            A5SH = computeAmplitude5SH(angle, x_s[j], y_s[j], x2[i], y2[i])
            AR = computeInitAmp(x_s[j], y_s[j], x2[i], y2[i], d_s[j], 'R', angle)
            A5SH = A5SH * AR[2]
        # Reflected wave on mid-slab discontinuity
        # Downgoing wave in upper oceanic crust
        for k1 in range(0, 2):
            # Upgoing wave in upper oceanic crust
            for k2 in range(0, 2):
                # Upgoing wave in continental crust
                for k3 in range(0, 2):
                    k = k1 * 1 + k2 * 2 + k3 * 4
                    angle = computeAngle3(x_s[j], y_s[j], d_s[j], x2[i], y2[i], wave[k1], wave[k2], wave[k3])
                    t3PSV[j, k] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x2[i], y2[i], \
                                  wave[k1], wave[k2], wave[k3])
                    if j == 0:
                        A3PSV[k] = computeAmplitude3PSV(angle, x_s[j], y_s[j], x2[i], y2[i], \
                                   wave[k1], wave[k2], wave[k3])
                        AR = computeInitAmp(x_s[j], y_s[j], x2[i], y2[i], d_s[j], 'R', angle)
                        if (k1 == 0):
                            A3PSV[k] = A3PSV[k] * AR[0]
                        else:
                            A3PSV[k] = A3PSV[k] * AR[1]
        # Reflected wave on Moho
        # Downgoing wave in upper oceanic crust
        for k1 in range(0, 2):
            # Downgoing wave in lower oceanic crust
            for k2 in range(0, 2):
                # Upgoing wave in lower oceanic crust
                for k3 in range(0, 2):
                    # Upgoing wave in upper oceanic crust
                    for k4 in range(0, 2):
                        # Upgoing wave in continental crust
                        for k5 in range(0, 2):
                            k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                            angle = computeAngle5(x_s[j], y_s[j], d_s[j], x2[i], y2[i], \
                                    wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                            t5PSV[j, k] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x2[i], y2[i], \
                                          wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                            if j == 0:
                                A5PSV[k] = computeAmplitude5PSV(angle, x_s[j], y_s[j], x2[i], y2[i], \
                                           wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                                AR = computeInitAmp(x_s[j], y_s[j], x2[i], y2[i], d_s[j], 'R', angle)
                                if (k1 == 0):
                                    A5PSV[k] = A5PSV[k] * AR[0]
                                else:
                                    A5PSV[k] = A5PSV[k] * AR[1]

    output.write('Time lag between direct waves\n')
    if ((abs(ASV / AP) >= 0.5) and (abs(ASV / AP) <= 2.0)):
        output.write('Amplitude of direct SV-wave: {}\n'.format(ASV / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (tS[1] - tP[1]) - (tS[0] - tP[0]), \
            (tS[2] - tP[2]) - (tS[0] - tP[0]), \
            (tS[3] - tP[3]) - (tS[0] - tP[0])))
        output.write('\n')
    if ((abs(ASH / AP) >= 0.5) and (abs(ASH / AP) <= 2.0)):
        output.write('Amplitude of direct SH-wave: {}\n'.format(ASH / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (tS[1] - tP[1]) - (tS[0] - tP[0]), \
            (tS[2] - tP[2]) - (tS[0] - tP[0]), \
            (tS[3] - tP[3]) - (tS[0] - tP[0])))
        output.write('\n')

    output.write('Time lag with direct P-wave\n')
    if ((abs(A3SH / AP) >= 0.5) and (abs(A3SH / AP) <= 2.0)):
        output.write('Amplitude of SH-wave (mid-slab): {}\n'.format(A3SH / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t3SH[1] - tP[1]) - (t3SH[0] - tP[0]), \
            (t3SH[2] - tP[2]) - (t3SH[0] - tP[0]), \
            (t3SH[3] - tP[3]) - (t3SH[0] - tP[0])))
        output.write('\n')
    if ((abs(A5SH / AP) >= 0.5) and (abs(A5SH / AP) <= 2.0)):
        output.write('Amplitude of SH-wave (Moho): {}\n'.format(A5SH / AP))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t5SH[1] - tP[1]) - (t5SH[0] - tP[0]), \
            (t5SH[2] - tP[2]) - (t5SH[0] - tP[0]), \
            (t5SH[3] - tP[3]) - (t5SH[0] - tP[0])))
        output.write('\n')
    # Reflected wave on mid-slab discontinuity
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Upgoing wave in upper oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in continental crust
            for k3 in range(0, 2):
                k = k1 * 1 + k2 * 2 + k3 * 4
                if ((abs(A3PSV[k] / AP) >= 0.5) and (abs(A3PSV[k] / AP) <= 2.0)):
                    output.write('Amplitude of ray {}{}{}: {}\n'.format(wave[k1], wave[k2], wave[k3], A3PSV[k] / AP))
                    output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                        (t3PSV[1, k] - tP[1]) - (t3PSV[0, k] - tP[0]), \
                        (t3PSV[2, k] - tP[2]) - (t3PSV[0, k] - tP[0]), \
                        (t3PSV[3, k] - tP[3]) - (t3PSV[0, k] - tP[0])))
                    output.write('\n')
    # Reflected wave on Moho
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Downgoing wave in lower oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in lower oceanic crust
            for k3 in range(0, 2):
                # Upgoing wave in upper oceanic crust
                for k4 in range(0, 2):
                    # Upgoing wave in continental crust
                    for k5 in range(0, 2):
                        k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                        if ((abs(A5PSV[k] / AP) >= 0.5) and (abs(A5PSV[k] / AP) <= 2.0)):
                            output.write('Amplitude of ray {}{}{}{}{}: {}\n'.format( \
                                wave[k1], wave[k2], wave[k3], wave[k4], wave[k5], A5PSV[k] / AP))
                            output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                                (t5PSV[1, k] - tP[1]) - (t5PSV[0, k] - tP[0]), \
                                (t5PSV[2, k] - tP[2]) - (t5PSV[0, k] - tP[0]), \
                                (t5PSV[3, k] - tP[3]) - (t5PSV[0, k] - tP[0])))
                            output.write('\n')

    output.write('Time lag with direct S-wave\n')
    if ((abs(A3SH / ASH) >= 0.5) and (abs(A3SH / ASH) <= 2.0)):
        output.write('Amplitude of SH-wave (mid-slab): {}\n'.format(A3SH / ASH))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t3SH[1] - tS[1]) - (t3SH[0] - tS[0]), \
            (t3SH[2] - tS[2]) - (t3SH[0] - tS[0]), \
            (t3SH[3] - tS[3]) - (t3SH[0] - tS[0])))
        output.write('\n')
    if ((abs(A5SH / ASH) >= 0.5) and (abs(A5SH / ASH) <= 2.0)):
        output.write('Amplitude of SH-wave (Moho): {}\n'.format(A5SH / ASH))
        output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
            (t5SH[1] - tS[1]) - (t5SH[0] - tS[0]), \
            (t5SH[2] - tS[2]) - (t5SH[0] - tS[0]), \
            (t5SH[3] - tS[3]) - (t5SH[0] - tS[0])))
        output.write('\n')
    # Reflected wave on mid-slab discontinuity
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Upgoing wave in upper oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in continental crust
            for k3 in range(0, 2):
                k = k1 * 1 + k2 * 2 + k3 * 4
                if ((abs(A3PSV[k] / ASV) >= 0.5) and (abs(A3PSV[k] / ASV) <= 2.0)):
                    output.write('Amplitude of ray {}{}{}: {}\n'.format(wave[k1], wave[k2], wave[k3], A3PSV[k] / ASV))
                    output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                        (t3PSV[1, k] - tS[1]) - (t3PSV[0, k] - tS[0]), \
                        (t3PSV[2, k] - tS[2]) - (t3PSV[0, k] - tS[0]), \
                        (t3PSV[3, k] - tS[3]) - (t3PSV[0, k] - tS[0])))
                    output.write('\n')
    # Reflected wave on Moho
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Downgoing wave in lower oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in lower oceanic crust
            for k3 in range(0, 2):
                # Upgoing wave in upper oceanic crust
                for k4 in range(0, 2):
                    # Upgoing wave in continental crust
                    for k5 in range(0, 2):
                        k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                        if ((abs(A5PSV[k] / ASV) >= 0.5) and (abs(A5PSV[k] / ASV) <= 2.0)):
                            output.write('Amplitude of ray {}{}{}{}{}: {}\n'.format( \
                                wave[k1], wave[k2], wave[k3], wave[k4], wave[k5], A5PSV[k] / ASV))
                            output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                                (t5PSV[1, k] - tS[1]) - (t5PSV[0, k] - tS[0]), \
                                (t5PSV[2, k] - tS[2]) - (t5PSV[0, k] - tS[0]), \
                                (t5PSV[3, k] - tS[3]) - (t5PSV[0, k] - tS[0])))
                            output.write('\n')

Close output file.

In [13]:
output.close()

The difference in time lags stays low for all the stations aligned along the strike of the plate boundary, even for tremor streaks travelling at 200 km/h. However, for the stations aligned along the dip of the plate boundary, the tremor streaks travelling at 200 km/h will cause a problem for the stations that are not immediately above the tremor source. The tremor streaks travelling at 100 km/h will cause a problem for the stations located more than 6 km updip of the tremor source. 